In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import datasets
import lightning as L

from lightning.pytorch.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader

from src.model.modeling_bonita import LitBonita

In [3]:
def train_valid_test_split(
    ds,
    split_ratio,
    seed: int,
):
    ds_train_test = ds.train_test_split(
        test_size=split_ratio[2] / sum(split_ratio), seed=seed
    )
    ds_train_valid = ds_train_test["train"].train_test_split(
        test_size=split_ratio[1] / (split_ratio[1] + split_ratio[0]), seed=seed
    )
    ds_train = ds_train_valid["train"]
    ds_valid = ds_train_valid["test"]
    ds_test = ds_train_test["test"]
    return ds_train, ds_valid, ds_test

In [4]:
SEED=42
DATASET_NAME = 'AutoML/bitabuse'
MINI_BATCH_SIZE=2
N_BATCH = 16
BASE_MODEL_NAME='Qwen/Qwen3-0.6B-Base'
EPOCHS=20
LEARNING_RATE = 5e-5
USE_BNTD=True
MAX_LENGTH=32
SPLITS = (1, 20, 79)
INFERENCE_SENTENCE_MAX_LENGTH=256
INFERENCE_SENTENCE_MIN_LENGTH=128
INFERENCE_SENTENCE_N_OVERLAP=1

In [5]:
L.seed_everything(SEED)

Seed set to 42


42

In [6]:
ds = datasets.load_dataset(DATASET_NAME, split="train")
def preprocessing(example):
    example['sentence_noisy'] = example['text'][:MAX_LENGTH]
    example['sentence'] = example['label'][:MAX_LENGTH]
    return example
ds = ds.map(preprocessing)
ds_train, ds_valid, ds_test = train_valid_test_split(
    ds, SPLITS, SEED
)


In [7]:
ds_valid = ds_valid.select(range(len(ds_train)))

In [8]:
dl_train, dl_valid, dl_test = DataLoader(ds_train, batch_size=MINI_BATCH_SIZE), DataLoader(ds_valid, batch_size=MINI_BATCH_SIZE), DataLoader(ds_test, batch_size=1)

In [9]:
lit_bind = LitBonita(
    base_model_name=BASE_MODEL_NAME,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    use_bntd=USE_BNTD,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

In [10]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/bonita',
    filename=f"{DATASET_NAME.split('/')[1]}/{BASE_MODEL_NAME.split('/')[1]}"+"-{epoch:02d}-{valid_loss:.4f}",
    every_n_epochs=1,
    save_top_k=-1,
)

In [11]:
trainer = L.Trainer(
    callbacks=[checkpoint_callback],
    precision='16-mixed',
    max_epochs=EPOCHS,
    enable_checkpointing=True,
    accumulate_grad_batches=N_BATCH
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(lit_bind, dl_train, dl_valid)

You are using a CUDA device ('NVIDIA GeForce RTX 5070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:658: Checkpoint directory C:\Users\jinwo\Documents\KROP\checkpoints\bonita exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params | Mode 
------------------------------------------
0 | bonita | Bonita | 596 M  | train
------------------------------------------
596 M     Trainable params
0         Non-trainable params
596 M     Total params
2,384.200 Total estimated model params size (MB)
1         Modules in train mode
427       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

In [8]:
lit_bonita = LitBonita.load_from_checkpoint(
    'checkpoints/bonita/bitabuse/Qwen3-0.6B-Base-epoch=03-valid_loss=0.0168.ckpt',
    base_model_name=BASE_MODEL_NAME,
    lr=LEARNING_RATE,
    epochs=EPOCHS,
    use_bntd=USE_BNTD,
    inference_sentence_max_length=INFERENCE_SENTENCE_MAX_LENGTH,
    inference_sentence_min_length=INFERENCE_SENTENCE_MIN_LENGTH,
    inference_sentence_n_overlap=INFERENCE_SENTENCE_N_OVERLAP
)

In [9]:
trainer = L.Trainer()

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
preds = trainer.predict(lit_bonita, dl_test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\jinwo\.virtualenvs\KROP-L3im0CPD\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [15]:
for batch in dl_test:
    print(batch['sentence'])

["here's one of your pass word [re"]
['immediately after that, i will g']
['claims to be elon musk, just wai']
['in case this does not occur - ev']
['this is insane they even know my']
['you have one day to pay.']
['i am ahacker who openedyour emai']
['when you have browsed the web pa']
['i had a threatening spyware infe']
['you actually have not one but tw']


In [14]:
preds

[["here's one of your pass word [re"],
 ['lmmediately after that, i will g'],
 ['claims to be alon musk, just wai'],
 ['in case this does not occur - ev'],
 ['this is insane they even know my'],
 ['you have one day to pay.'],
 ['i am ahacker who openedyour emal'],
 ['when you have browsed the wep pa'],
 ['i had a threatening spyware inre'],
 ['you actoally have not one but tw']]